In [1]:
# noexport

import os
os.system('export_notebook identify_domain.ipynb')

0

In [1]:
from tmilib import *

In [3]:
#user = get_test_users()[0]
#print user

In [4]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print tab_focus_times[0].keys()

In [5]:
#print active_second_to_domain_id.keys()[:10]

In [7]:
#seen_domains_only_real = get_seen_urls_and_domains_real_and_history()['seen_domains_only_real']

In [11]:
@memoized
def get_recently_seen_domain_stats_for_user_v2(user):
  ordered_visits = get_history_ordered_visits_corrected_for_user(user)
  ordered_visits = exclude_bad_visits(ordered_visits)
  active_domain_at_time = get_active_domain_at_time_for_user(user)
  active_seconds_set = set(get_active_insession_seconds_for_user(user))
  #active_second_to_domain_id = get_active_second_to_domain_id_for_user(user)
  active_second_to_domain_id = {int(k):v for k,v in get_active_second_to_domain_id_for_user(user).viewitems()}
  visit_id_to_domain_id = {}
  stats = Counter()
  for idx,visit in enumerate(ordered_visits):
    if idx+1 >= len(ordered_visits):
      break
    next_visit = ordered_visits[idx+1]
    cur_domain = url_to_domain(visit['url'])
    cur_domain_id = domain_to_id(cur_domain)
    next_domain = url_to_domain(next_visit['url'])
    next_domain_id = domain_to_id(next_domain)
    cur_time_sec = int(round(visit['visitTime'] / 1000.0))
    next_time_sec = int(round(next_visit['visitTime'] / 1000.0))
    
    cur_visit_id = visit['visitId']
    visit_id_to_domain_id[cur_visit_id] = cur_domain_id
    
    next_refer_id = next_visit['referringVisitId']
    referring_domain_id = visit_id_to_domain_id.get(next_refer_id, -1)
    
    if cur_time_sec > next_time_sec:
      continue
    
    for time_sec in xrange(cur_time_sec, next_time_sec+1):
      if time_sec not in active_seconds_set:
        continue
      ref_domain_id = active_second_to_domain_id[time_sec]
      stats['total'] += 1
      
      if cur_domain_id == ref_domain_id and referring_domain_id == ref_domain_id:
        stats['both cur and referring are correct'] += 1
      elif referring_domain_id == ref_domain_id:
        stats['referring_domain_correct'] += 1
        if referring_domain_id == next_domain_id:
          stats['referring_domain_eq_next'] += 1
      elif cur_domain_id == ref_domain_id:
        stats['cur domain correct'] += 1
  return stats

In [12]:
total_stats = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user_v2(user).iteritems():
    total_stats[k] += v


In [13]:
print {k:v/float(total_stats['total']) for k,v in total_stats.items()}

{'referring_domain_eq_next': 0.03962201889338221, 'both cur and referring are correct': 0.2729347038691897, 'total': 1.0, 'referring_domain_correct': 0.04147766958476675, 'cur domain correct': 0.4712744246409016}


In [10]:
@memoized
def get_recently_seen_domain_stats_for_user(user):
  ordered_visits = get_history_ordered_visits_corrected_for_user(user)
  ordered_visits = exclude_bad_visits(ordered_visits)
  active_domain_at_time = get_active_domain_at_time_for_user(user)
  active_seconds_set = set(get_active_insession_seconds_for_user(user))
  #active_second_to_domain_id = get_active_second_to_domain_id_for_user(user)
  active_second_to_domain_id = {int(k):v for k,v in get_active_second_to_domain_id_for_user(user).viewitems()}
  recently_seen_domain_ids = [-1]*100
  seen_domain_ids_set = set()
  stats = Counter()
  nomatch_domains = Counter()
  tabbed_back_domains_first = Counter()
  tabbed_back_domains_second = Counter()
  tabbed_back_domains_all = Counter()
  distractors_list = [domain_to_id(x) for x in ['www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com']]
  distractors = set(distractors_list)
  most_recent_distractor = distractors_list[0]
  for idx,visit in enumerate(ordered_visits):
    if idx+1 >= len(ordered_visits):
      break
    next_visit = ordered_visits[idx+1]
    cur_domain = url_to_domain(visit['url'])
    cur_domain_id = domain_to_id(cur_domain)
    if cur_domain_id in distractors:
      most_recent_distractor = cur_domain_id
    if cur_domain_id != recently_seen_domain_ids[-1]:
      if cur_domain_id in seen_domain_ids_set:
        recently_seen_domain_ids.remove(cur_domain_id)
      seen_domain_ids_set.add(cur_domain_id)
      recently_seen_domain_ids.append(cur_domain_id)
    next_domain = url_to_domain(next_visit['url'])
    next_domain_id = domain_to_id(next_domain)
    cur_time_sec = int(round(visit['visitTime'] / 1000.0))
    next_time_sec = int(round(next_visit['visitTime'] / 1000.0))
    
    if cur_time_sec > next_time_sec:
      continue
    
    for time_sec in xrange(cur_time_sec, next_time_sec+1):
      if time_sec not in active_seconds_set:
        continue
      ref_domain_id = active_second_to_domain_id[time_sec]
      stats['total'] += 1
      if most_recent_distractor == ref_domain_id:
        stats['most_recent_distractor_total'] += 1
        if cur_domain_id == ref_domain_id:
          stats['most_recent_distractor_curdomain'] += 1
        elif cur_domain_id == next_domain_id:
          stats['most_recent_distractor_nextdomain'] += 1
        else:
          stats['most_recent_distractor_some_prev_domain'] += 1
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
        else:
          stats['first correct only'] += 1
        continue
      if next_domain_id == ref_domain_id:
        stats['next correct only'] += 1
        continue
      stats['both incorrect'] += 1
      found_match = False
      ref_domain = id_to_domain(ref_domain_id)
      for i in range(1,101):
        if recently_seen_domain_ids[-1-i] == ref_domain_id:
          stats['nth previous correct ' + str(abs(i))] += 1
          stats['some previous among past 100 correct'] += 1
          found_match = True
          tabbed_back_domains_all[ref_domain] += 1
          if i == 1:
            tabbed_back_domains_first[ref_domain] += 1
          if i == 2:
            tabbed_back_domains_second[ref_domain] += 1
          break
      if not found_match:
        ref_domain = id_to_domain(ref_domain_id)
        if ref_domain == 'newtab':
          stats['newtab'] += 1
          continue
        stats['no match found'] += 1
        nomatch_domains[id_to_domain(ref_domain_id)] += 1
      '''
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
          continue
        else:
          stats['first correct only'] += 1
          continue
      else:
        if next_domain_id == cur_domain_id:
          stats['both incorrect'] += 1
          found_match = False
          for i in range(1,101):
            if recently_seen_domain_ids[-1-i] == ref_domain_id:
              stats['nth previous correct ' + str(abs(i))] += 1
              stats['some previous among past 100 correct'] += 1
              found_match = True
              break
          if not found_match:
            ref_domain = id_to_domain(ref_domain_id)
            if ref_domain == 'newtab':
              stats['newtab'] += 1
              continue
            stats['no match found'] += 1
            nomatch_domains[id_to_domain(ref_domain_id)] += 1
          continue
        if next_domain_id == ref_domain_id:
          stats['next correct only'] += 1
          continue
      '''
  return {
    'stats': stats,
    'nomatch_domains': nomatch_domains,
    'tabbed_back_domains_all': tabbed_back_domains_all,
    'tabbed_back_domains_first': tabbed_back_domains_first,
    'tabbed_back_domains_second': tabbed_back_domains_second,
  }

In [11]:
total_nomatch_domains = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user(user)['nomatch_domains'].viewitems():
    total_nomatch_domains[k] += v


In [12]:
#print total_nomatch_domains
print_counter(total_nomatch_domains)

bookmarks 8534
tmi.stanford.edu 4943
www.pandora.com 3017
klbibkeccnjlkjkiokjodocebajanakg 2674
www.mturk.com 2254
 2056
downloads 1889
history 1350
www.google.com 1280
settings 1152
ww2.unipark.de 1038
www.youtube.com 920
mprlab327.webfactional.com 874
extensions 869
www.amazon.com 748
www.heatherridley.com 712
features.texasmonthly.com 633
www.facebook.com 628
knightlytales.wikidot.com 605
www.reenaahluwalia.com 591
www.nextmuni.com 578
www.reddit.com 578
local-ntp 575
apps.facebook.com 551
www.surveymonkey.com 533
vassarpsych.az1.qualtrics.com 511
reader.chegg.com 507
mail.google.com 507
armorgames.com 491
usccollege.qualtrics.com 484
uky.az1.qualtrics.com 472
uwmadison.co1.qualtrics.com 465
survey.az1.qualtrics.com 456
hbs.qualtrics.com 429
drive.google.com 425
www.flashx.host 400
spellout.net 376
rnd-testing.wonderliconline.com 369
uscpharmacy.az1.qualtrics.com 345
www.kickstarter.com 333
www.animefillerlist.com 325
gcat.dot.state.oh.us 318
koreabus.az1.qualtrics.com 317
128.151.1

In [13]:
total_tabbed_back_domains_all = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user(user)['tabbed_back_domains_all'].viewitems():
    total_tabbed_back_domains_all[k] += v

In [14]:
#[domain_to_id(x) for x in ['www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com']]

In [15]:
#sumkeys(total_tabbed_back_domains_all, 'www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com')

In [16]:
print sum(total_tabbed_back_domains_all.values())

2382221


In [17]:
#728458.0/2382221

In [18]:
print_counter(total_tabbed_back_domains_all)

www.mturk.com 259154
apps.facebook.com 157564
www.facebook.com 154560
www.reddit.com 81526
www.youtube.com 75654
www.google.com 45171
www.twitch.tv 40482
docs.google.com 37875
mail.google.com 37240
www.netflix.com 34310
www.amazon.com 34070
allmyvideos.net 30749
www.tumblr.com 19361
www.mturkgrind.com 19318
www.amzreviewtrader.com 16451
www.speechpad.com 15963
hitgrabber.net 14552
www.allmyvideos.net 12324
armorgames.com 11310
wharton.qualtrics.com 11022
bbs2.ruliweb.daum.net 11019
mturkforum.com 10942
www.hulu.com 10858
iu.co1.qualtrics.com 10688
forums.animez.to 10633
agar.io 10597
yalesurvey.qualtrics.com 10481
us-mg5.mail.yahoo.com 10278
www.addictinggames.com 10234
upenn.co1.qualtrics.com 9938
www.marketwatch.com 9847
kellogg.qualtrics.com 9680
mail.aol.com 9256
myanimelist.net 8551
 8429
mprlab327.webfactional.com 8092
saintleo.learningstudio.com 7694
unc.az1.qualtrics.com 7503
www.swagbucks.com 7282
hbs.qualtrics.com 7173
washington.co1.qualtrics.com 7036
nyu.qualtrics.com 6982


In [19]:
#print_counter(total_tabbed_back_domains_first)

In [20]:
'''
total_active_seconds = 0

for user in get_test_users():
  total_active_seconds += len(get_active_seconds_for_user(user))
  #for span in get_active_seconds_for_user(user)
print total_active_seconds
'''

'\ntotal_active_seconds = 0\n\nfor user in get_test_users():\n  total_active_seconds += len(get_active_seconds_for_user(user))\n  #for span in get_active_seconds_for_user(user)\nprint total_active_seconds\n'

In [27]:
#for user in get_test_users():
#  print user
#  print (get_recently_seen_domain_stats_for_user(user)['stats'])

UrZvGUsI8N
Counter({'total': 110232, 'first and next equal and correct': 55530, 'most_recent_distractor_total': 39133, 'most_recent_distractor_curdomain': 34872, 'first correct only': 30604, 'both incorrect': 17975, 'some previous among past 100 correct': 16416, 'next correct only': 6123, 'nth previous correct 2': 5320, 'nth previous correct 1': 4733, 'most_recent_distractor_some_prev_domain': 2886, 'nth previous correct 4': 1993, 'nth previous correct 3': 1926, 'most_recent_distractor_nextdomain': 1375, 'nth previous correct 5': 924, 'no match found': 912, 'newtab': 647, 'nth previous correct 6': 528, 'nth previous correct 7': 334, 'nth previous correct 25': 148, 'nth previous correct 11': 142, 'nth previous correct 18': 87, 'nth previous correct 19': 71, 'nth previous correct 14': 64, 'nth previous correct 30': 60, 'nth previous correct 8': 56, 'nth previous correct 9': 7, 'nth previous correct 15': 5, 'nth previous correct 10': 4, 'nth previous correct 35': 4, 'nth previous correct 

In [2]:
@memoized
def get_recently_seen_domain_stats_for_user_v3(user):
  ordered_visits = get_history_ordered_visits_corrected_for_user(user)
  ordered_visits = exclude_bad_visits(ordered_visits)
  active_domain_at_time = get_active_domain_at_time_for_user(user)
  active_seconds_set = set(get_active_insession_seconds_for_user(user))
  #active_second_to_domain_id = get_active_second_to_domain_id_for_user(user)
  active_second_to_domain_id = {int(k):v for k,v in get_active_second_to_domain_id_for_user(user).viewitems()}
  recently_seen_domain_ids = [-1]*100
  seen_domain_ids_set = set()
  stats = Counter()
  nomatch_domains = Counter()
  tabbed_back_domains_first = Counter()
  tabbed_back_domains_second = Counter()
  tabbed_back_domains_all = Counter()
  distractors_list = [domain_to_id(x) for x in ['www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com']]
  distractors = set(distractors_list)
  most_recent_distractor = distractors_list[0]
  for idx,visit in enumerate(ordered_visits):
    if idx+1 >= len(ordered_visits):
      break
    next_visit = ordered_visits[idx+1]
    cur_domain = url_to_domain(visit['url'])
    cur_domain_id = domain_to_id(cur_domain)
    if cur_domain_id in distractors:
      most_recent_distractor = cur_domain_id
    if cur_domain_id != recently_seen_domain_ids[-1]:
      if cur_domain_id in seen_domain_ids_set:
        recently_seen_domain_ids.remove(cur_domain_id)
      seen_domain_ids_set.add(cur_domain_id)
      recently_seen_domain_ids.append(cur_domain_id)
    next_domain = url_to_domain(next_visit['url'])
    next_domain_id = domain_to_id(next_domain)
    cur_time_sec = int(round(visit['visitTime'] / 1000.0))
    next_time_sec = int(round(next_visit['visitTime'] / 1000.0))
    
    if cur_time_sec > next_time_sec:
      continue
    
    for time_sec in xrange(cur_time_sec, next_time_sec+1):
      if time_sec not in active_seconds_set:
        continue
      ref_domain_id = active_second_to_domain_id[time_sec]
      stats['total'] += 1
      if most_recent_distractor == ref_domain_id:
        stats['most_recent_distractor_total'] += 1
        if cur_domain_id == ref_domain_id:
          stats['most_recent_distractor_curdomain'] += 1
        elif cur_domain_id == next_domain_id:
          stats['most_recent_distractor_nextdomain'] += 1
        else:
          stats['most_recent_distractor_some_prev_domain'] += 1
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
        else:
          stats['first correct only'] += 1
        continue
      #if next_domain_id == ref_domain_id:
      #  stats['next correct only'] += 1
      #  continue
      stats['both incorrect'] += 1
      found_match = False
      ref_domain = id_to_domain(ref_domain_id)
      for i in range(1,101):
        if recently_seen_domain_ids[-1-i] == ref_domain_id:
          stats['nth previous correct ' + str(abs(i))] += 1
          stats['some previous among past 100 correct'] += 1
          found_match = True
          tabbed_back_domains_all[ref_domain] += 1
          if i == 1:
            tabbed_back_domains_first[ref_domain] += 1
          if i == 2:
            tabbed_back_domains_second[ref_domain] += 1
          break
      if not found_match:
        ref_domain = id_to_domain(ref_domain_id)
        #if ref_domain == 'newtab':
        #  stats['newtab'] += 1
        #  continue
        stats['no match found'] += 1
        nomatch_domains[id_to_domain(ref_domain_id)] += 1
      '''
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
          continue
        else:
          stats['first correct only'] += 1
          continue
      else:
        if next_domain_id == cur_domain_id:
          stats['both incorrect'] += 1
          found_match = False
          for i in range(1,101):
            if recently_seen_domain_ids[-1-i] == ref_domain_id:
              stats['nth previous correct ' + str(abs(i))] += 1
              stats['some previous among past 100 correct'] += 1
              found_match = True
              break
          if not found_match:
            ref_domain = id_to_domain(ref_domain_id)
            if ref_domain == 'newtab':
              stats['newtab'] += 1
              continue
            stats['no match found'] += 1
            nomatch_domains[id_to_domain(ref_domain_id)] += 1
          continue
        if next_domain_id == ref_domain_id:
          stats['next correct only'] += 1
          continue
      '''
  return {
    'stats': stats,
    'nomatch_domains': nomatch_domains,
    'tabbed_back_domains_all': tabbed_back_domains_all,
    'tabbed_back_domains_first': tabbed_back_domains_first,
    'tabbed_back_domains_second': tabbed_back_domains_second,
  }

In [3]:
#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41231, 'nth previous correct 1': 31202, 'next correct only': 23569, 'no match found': 10432, 'nth previous correct 2': 3311, 'nth previous correct 3': 1635, 'nth previous correct 4': 905, 'nth previous correct 5': 862, 'nth previous correct 6': 545, 'nth previous correct 7': 412, 'nth previous correct 8': 357, 'nth previous correct 9': 269, 'nth previous correct 10': 259, 'nth previous correct 13': 234, 'nth previous correct 11': 229, 'nth previous correct 12': 190, 'nth previous correct 15': 183, 'nth previous correct 14': 140, 'nth previous correct 17': 139, 'nth previous correct 20': 95, 'nth previous correct 16': 90, 'nth previous correct 19': 88, 'nth previous correct 18': 86})

total_stats = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user_v3(user)['stats'].viewitems():
    total_stats[k] += v
#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41136, 'nth previous correct 2': 31202, 'next correct only': 23569, 'no match found': 10527, 'nth previous correct 3': 3311, 'nth previous correct 4': 1635, 'nth previous correct 5': 905, 'nth previous correct 6': 862, 'nth previous correct 7': 545, 'nth previous correct 8': 412, 'nth previous correct 9': 357, 'nth previous correct 10': 269, 'nth previous correct 11': 259, 'nth previous correct 14': 234, 'nth previous correct 12': 229, 'nth previous correct 13': 190, 'nth previous correct 16': 183, 'nth previous correct 15': 140, 'nth previous correct 18': 139, 'nth previous correct 17': 90, 'nth previous correct 20': 88, 'nth previous correct 19': 86})

In [4]:
print_counter(total_stats)

total 14806666
first and next equal and correct 6588587
most_recent_distractor_total 4978110
first correct only 4430669
most_recent_distractor_curdomain 4226119
both incorrect 3787410
some previous among past 100 correct 3598307
nth previous correct 1 1646024
nth previous correct 2 674420
most_recent_distractor_some_prev_domain 621347
nth previous correct 3 374076
nth previous correct 4 218390
no match found 189103
most_recent_distractor_nextdomain 130644
nth previous correct 5 128902
nth previous correct 6 102701
nth previous correct 7 71187
nth previous correct 8 58600
nth previous correct 9 43291
nth previous correct 10 42584
nth previous correct 11 29831
nth previous correct 12 23107
nth previous correct 13 22927
nth previous correct 15 13742
nth previous correct 16 12968
nth previous correct 17 12869
nth previous correct 14 12776
nth previous correct 18 11522
nth previous correct 19 7643
nth previous correct 21 7236
nth previous correct 20 6725
nth previous correct 22 6291
nth pre

In [5]:
def sumkeys(d, *args):
  return sum(d.get(x, 0.0) for x in args)

In [6]:
norm = {k:float(v)/total_stats['total'] for k,v in total_stats.viewitems()}
print 'select prev gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only')
print 'prev or next gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only')
print 'prev or next or newtab gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')
for i in range(1, 101):
  sumprev = sum([norm.get('nth previous correct '+str(x),0.0) for x in range(i+1)])
  print 'prev or next or past ' + str(i), sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')+sumprev

select prev gets answer correct 0.74420912851
prev or next gets answer correct 0.74420912851
prev or next or newtab gets answer correct 0.74420912851
prev or next or past 1 0.855376895785
prev or next or past 2 0.900925299456
prev or next or past 3 0.926189325808
prev or next or past 4 0.940938763662
prev or next or past 5 0.949644437174
prev or next or past 6 0.95658056986
prev or next or past 7 0.961388336848
prev or next or past 8 0.965346013748
prev or next or past 9 0.968269764443
prev or next or past 10 0.971145766373
prev or next or past 11 0.973160467049
prev or next or past 12 0.974721047939
prev or next or past 13 0.976269472142
prev or next or past 14 0.977132326751
prev or next or past 15 0.978060422245
prev or next or past 16 0.978936243986
prev or next or past 17 0.97980537955
prev or next or past 18 0.980583542575
prev or next or past 19 0.981099729
prev or next or past 20 0.981553916324
prev or next or past 21 0.98204261513
prev or next or past 22 0.982467491331
prev or

In [25]:
#print norm['most_recent_distractor_total']
#print norm['most_recent_distractor_curdomain']
#print norm['most_recent_distractor_nextdomain']
#print norm['most_recent_distractor_some_prev_domain']